In [1]:
from selenium import webdriver
import pandas as pd

In [2]:
def go_to_offer(idx):
    elements = driver.find_elements_by_css_selector("span.offer-item-title")
    elements[idx].click()
    

def back_to_list():
    driver.execute_script("""
    window.history.back();
    """)

    
def go_to_next_page():
    element = driver.find_element_by_css_selector("li.pager-next")
    element.click()

    
def scroll_to_normal_offers():
    driver.execute_script("""
    const offset = window.pageYOffset
    window.scrollTo(0,offset + 1600)
    """)

    
def scroll_to_offer():
    driver.execute_script("""
    const offset = window.pageYOffset
    window.scrollTo(0,offset + 225)
    """)

    
def read_offer():
    location = ""
    while location == "":
        location = driver.find_element_by_css_selector('a.css-12hd9gg').text
 
    prize = driver.find_element_by_css_selector('div.css-1vr19r7').text.rsplit(maxsplit = 1)[0].replace(' ', '')
    prize_per_meter = driver.find_element_by_css_selector('div.css-18q4l99').text.rsplit(maxsplit = 1)[0].replace(' ', '')
    properties = driver.find_element_by_css_selector("h2 + div > ul").find_elements_by_tag_name('li')
    title = driver.find_element_by_tag_name('h1').text
    
    prop_dict = {
        'Tytul' : title,
        'Lokalizacja' : location,
        'Cena' : prize,
        'Cena_za_metr' : prize_per_meter, 
    }
    
    for prop in properties:
        k,v = prop.text.split(':')       
        prop_dict[k.strip()] = v.strip()
    
    return prop_dict

In [3]:
driver = webdriver.Chrome("./chromedriver")

driver.get("https://www.otodom.pl/sprzedaz/mieszkanie")
driver.set_window_size(800, 1080)
driver.implicitly_wait(10)

offers = []
pages = 5
offer_on_page = 24

for i in range(pages):
    scroll_to_normal_offers()
    for j in range(offer_on_page):
        try:
            go_to_offer(j+3) # to exclude promotion offer
        except Exception as error:
            print(error)
        offers.append(read_offer())
        back_to_list()
        scroll_to_offer()
    go_to_next_page()

driver.close()

In [12]:
df = pd.DataFrame(offers)
df.head(10)

,Tytul,Lokalizacja,Cena,Cena_za_metr,Powierzchnia,Liczba pokoi,Rynek,Piętro,Liczba pięter,Okna,Rodzaj zabudowy,Czynsz,Forma własności,Ogrzewanie,Rok budowy,Materiał budynku,Stan wykończenia,Dostępne od
0,Mieszkanie po generalnym remoncie!,"Bydgoszcz, Bartodzieje",269000,7231,"37,20 m²",2,wtórny,4,11,plastikowe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Rzeszów, mieszkanie na sprzedaż, Baranówka","Rzeszów, podkarpackie",264000,5116,"51,60 m²",3,wtórny,3,NaN,plastikowe,blok,490 zł,pełna własność,NaN,NaN,NaN,NaN,NaN
2,"Mieszkanie, 31 m², Kraków","Kraków, Nowa Huta, Czyżyny, os. Kościuszkowskie",333000,10742,31 m²,1,wtórny,5,10,plastikowe,blok,279 zł,NaN,miejskie,1970,NaN,NaN,NaN
3,"Borek Fałęcki, 2 niezależne pokoje","Kraków, Łagiewniki-Borek Fałęcki, Borek Fałęck...",233000,6853,34 m²,2,wtórny,2,3,plastikowe,kamienica,200 zł,NaN,NaN,1920,cegła,NaN,NaN
4,"Mieszkanie, 36,45 m², Kraków","Kraków, Bieńczyce, os. Kazimierzowskie",302500,8299,"36,45 m²",2,wtórny,10,10,plastikowe,blok,270 zł,NaN,miejskie,NaN,NaN,do remontu,NaN
5,"2 pokoje + balkon, bez prowizji -ceny Dewelopera","Wieliczka, wielicki, małopolskie",279600,7249,"38,57 m²",2,pierwotny,2,4,plastikowe,blok,NaN,NaN,NaN,2021,NaN,NaN,NaN
6,"Dom Marzeń III Etap, mieszkanie 31","ul. Jana Pawła II, Piaseczno, piaseczyński, ma...",434979,7650,"56,86 m²",3,pierwotny,2,5,NaN,NaN,NaN,NaN,NaN,2019,NaN,do wykończenia,NaN
7,Nowe 3 pokoje + narożny ogródek 97m2,"Wieliczka, wielicki, małopolskie",353600,6950,"50,88 m²",3,pierwotny,parter,4,plastikowe,blok,NaN,NaN,NaN,2021,NaN,NaN,NaN
8,Nowe mieszkanie od dewelopera/Olimpia Port,"Wrocław, Psie Pole, Ferdynanda Magellana",513000,9189,"55,83 m²",3,wtórny,1,3,NaN,apartamentowiec,NaN,pełna własność,miejskie,2019,pustak,NaN,NaN
14,"Dom Marzeń III Etap, mieszkanie 30","ul. Jana Pawła II, Piaseczno, piaseczyński, ma...",463314,7400,"62,61 m²",3,pierwotny,2,5,NaN,NaN,NaN,NaN,NaN,2019,NaN,do wykończenia,NaN


In [13]:
# save DataFrame to csv file
df.to_csv("OtoDom.csv", sep=";")